This program works  9 April 2025

In [ ]:
"""
# Materials Property Explorer and Visualization
# ===========================================
#
# This project introduces students to accessing and visualizing data from the Materials Project API.
# Students will learn to:
# 1. Connect to the Materials Project API
# 2. Query for materials with specific properties
# 3. Visualize relationships between material properties using interactive plots
# 4. Explore structure-property relationships using crystal-toolkit for 3D structure visualization
"""

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mp_api.client import MPRester
import plotly.express as px
import plotly.graph_objects as go
from pymatgen.core import Structure
import crystal_toolkit
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

# Set your API key here
API_KEY = "Your API Key Goes here"  # Students will replace with their actual API key

def explore_oxide_materials():
    """
    Query for oxide materials from the Materials Project API and visualize the results.
    """
    print("Materials Property Explorer")
    print("==========================")
    
    print("\nExploring oxide semiconductors...")
    
    # Connect to Materials Project with API key
    with MPRester(API_KEY) as mpr:
        # Query for oxide semiconductors with band gaps between 0.5 and 4.0 eV
        # Only using fields explicitly listed in the available fields from the error message
        docs = mpr.materials.summary.search(
            elements=["O"],
            band_gap=(0.5, 0.9),
            fields=[
                "material_id", 
                "formula_pretty", 
                "structure", 
                "band_gap", 
                "density", 
                "energy_above_hull", 
                "formation_energy_per_atom",
                "volume", 
                "symmetry"
            ]
        )
        
        # Extract data to a list of dictionaries
        data = []
        for doc in docs:
            # Get crystal system from symmetry object if available
            try:
                crystal_system = doc.symmetry.crystal_system if hasattr(doc, 'symmetry') else "Unknown"
            except:
                crystal_system = "Unknown"
                
            data.append({
                "material_id": doc.material_id,
                "formula": doc.formula_pretty,
                "band_gap": doc.band_gap,
                "density": doc.density,
                "stability": doc.energy_above_hull,
                "formation_energy": doc.formation_energy_per_atom,
                "volume": doc.volume,
                "crystal_system": crystal_system,
                "structure": doc.structure
            })
        
        # Convert to DataFrame
        df = pd.DataFrame(data)
        
        print(f"Found {len(df)} oxide semiconductors")
        print(df[['material_id', 'formula', 'band_gap', 'stability']].head())
        
        # Create visualizations
        print("\nCreating visualizations...")
        
        # 1. Band gap vs. formation energy scatter plot using matplotlib
        plt.figure(figsize=(10, 6))
        scatter = plt.scatter(
            df['formation_energy'], 
            df['band_gap'], 
            c=df['stability'],
            cmap='viridis',
            alpha=0.7
        )
        plt.colorbar(scatter, label='Energy Above Hull (eV)')
        plt.xlabel('Formation Energy (eV/atom)')
        plt.ylabel('Band Gap (eV)')
        plt.title('Band Gap vs. Formation Energy')
        plt.grid(True, alpha=0.3)
        plt.savefig('bandgap_vs_formation_energy.png')
        plt.show()
        
        # 2. Interactive plotly visualizations
        # Band gap vs. formation energy
        fig = px.scatter(
            df, 
            x='formation_energy', 
            y='band_gap',
            color='stability',
            hover_name='formula',
            hover_data=['material_id'],
            labels={
                'formation_energy': 'Formation Energy (eV/atom)',
                'band_gap': 'Band Gap (eV)',
                'stability': 'Energy Above Hull (eV)'
            },
            title='Band Gap vs. Formation Energy'
        )
        fig.update_layout(template='plotly_white')
        fig.show()
        
        # Band gap vs. density by crystal system
        fig = px.scatter(
            df, 
            x='density', 
            y='band_gap',
            color='crystal_system',
            hover_name='formula',
            hover_data=['material_id', 'stability'],
            size='volume',
            size_max=20,
            labels={
                'density': 'Density (g/cm³)',
                'band_gap': 'Band Gap (eV)',
                'crystal_system': 'Crystal System',
                'volume': 'Volume (Å³)'
            },
            title='Band Gap vs. Density by Crystal System'
        )
        fig.update_layout(template='plotly_white')
        fig.show()
        
        # Distribution of band gaps
        fig = px.histogram(
            df, 
            x='band_gap',
            nbins=20,
            labels={'band_gap': 'Band Gap (eV)', 'count': 'Number of Materials'},
            title='Distribution of Band Gaps'
        )
        fig.update_layout(template='plotly_white')
        fig.show()
        
        # Find interesting materials
        print("\nFinding interesting materials...")
        
        # Materials with highest band gaps
        high_gap_materials = df.sort_values('band_gap', ascending=False).head(3)
        print("\nMaterials with highest band gaps:")
        print(high_gap_materials[['formula', 'material_id', 'band_gap', 'stability']])
        
        # Most stable materials with reasonable band gaps
        stable_materials = df[df['band_gap'] > 1.0].sort_values('stability').head(3)
        print("\nMost stable materials with band gap > 1 eV:")
        print(stable_materials[['formula', 'material_id', 'band_gap', 'stability']])
        
        # Visualize crystal structures
        # Visualize structure of the material with highest band gap
        if not high_gap_materials.empty:
            top_material = high_gap_materials.iloc[0]
            print(f"\nVisualizing structure of material with highest band gap:")
            print(f"{top_material['formula']} ({top_material['material_id']})")
            print(f"Band gap: {top_material['band_gap']:.2f} eV, Stability: {top_material['stability']:.3f} eV")
            visualize_structure(top_material['structure'], top_material['formula'])
        
        # Visualize structure of the most stable material
        if not stable_materials.empty:
            best_material = stable_materials.iloc[0]
            print(f"\nVisualizing structure of most stable material:")
            print(f"{best_material['formula']} ({best_material['material_id']})")
            print(f"Band gap: {best_material['band_gap']:.2f} eV, Stability: {best_material['stability']:.3f} eV")
            visualize_structure(best_material['structure'], best_material['formula'])
        
        print("\nExploration complete!")
        
        return df

def explore_materials_by_chemsys(chemsys="Si-O"):
    """
    Query materials by chemical system (e.g., Si-O) and visualize the results.
    This follows the pattern of the second provided example.
    
    Args:
        chemsys: Chemical system (e.g., "Si-O", "Fe-O")
    """
    print(f"\nExploring {chemsys} materials...")
    
    # Connect to Materials Project with API key
    with MPRester(API_KEY) as mpr:
        # Query for materials in the specified chemical system
        docs = mpr.materials.summary.search(
            chemsys=chemsys,
            fields=[
                "material_id", 
                "formula_pretty", 
                "structure", 
                "band_gap", 
                "density", 
                "energy_above_hull", 
                "formation_energy_per_atom",
                "volume"
            ]
        )
        
        # Convert results to pandas DataFrame
        data = {
            "material_id": [doc.material_id for doc in docs],
            "formula": [doc.formula_pretty for doc in docs],
            "band_gap": [doc.band_gap for doc in docs],
            "formation_energy": [doc.formation_energy_per_atom for doc in docs],
            "volume": [doc.volume for doc in docs],
            "density": [doc.density for doc in docs],
            "stability": [doc.energy_above_hull for doc in docs],
            "structure": [doc.structure for doc in docs]
        }
        df = pd.DataFrame(data)
        
        print(f"Found {len(df)} {chemsys} materials")
        
        # Create scatter plot using matplotlib
        plt.figure(figsize=(10, 6))
        plt.scatter(df["formation_energy"], df["band_gap"], alpha=0.7)
        plt.xlabel("Formation Energy (eV/atom)")
        plt.ylabel("Band Gap (eV)")
        plt.title(f"Band Gap vs. Formation Energy for {chemsys} Materials")
        plt.grid(True, alpha=0.3)
        plt.savefig(f"{chemsys}_bandgap_vs_formation_energy.png")
        plt.show()
        
        # Visualize most stable material
        if not df.empty:
            stable_material = df.sort_values('stability').iloc[0]
            print(f"\nVisualizing structure of most stable {chemsys} material:")
            print(f"{stable_material['formula']} ({stable_material['material_id']})")
            print(f"Stability: {stable_material['stability']:.3f} eV")
            visualize_structure(stable_material['structure'], stable_material['formula'])
        
        return df

def visualize_structure(structure, title=None):
    """
    Visualize a crystal structure using crystal-toolkit
    
    Args:
        structure: Pymatgen Structure object
        title: Title for the visualization
    """
    # Display the structure directly - crystal-toolkit will render it
    if title:
        print(f"\n{title} Structure:")
    
    # In Jupyter notebook environment, this will create an interactive 3D visualization
    # Adding a small delay can help ensure the visualization loads properly
    import time
    time.sleep(0.5)  # Small delay to ensure proper initialization
    
    # Display the structure
    display(structure)
    
    # Print additional structure information
    print(f"Formula: {structure.composition.reduced_formula}")
    try:
        print(f"Space Group: {structure.get_space_group_info()}")
    except:
        print("Space Group: Unable to determine")
    print(f"Lattice Parameters: a={structure.lattice.a:.4f}, b={structure.lattice.b:.4f}, c={structure.lattice.c:.4f}")
    print(f"Lattice Angles: α={structure.lattice.alpha:.2f}°, β={structure.lattice.beta:.2f}°, γ={structure.lattice.gamma:.2f}°")
    print(f"Volume: {structure.volume:.2f} Å³")
    print(f"Density: {structure.density:.4f} g/cm³")
    
    # Save structure to CIF file if needed
    filename = f"{structure.composition.reduced_formula}.cif"
    structure.to(filename)
    print(f"Saved structure to {filename}")

def main():
    """Main function to run the exploration"""
    # Demo of oxide materials exploration
    print("Running Materials Property Explorer")
    print("==================================")
    print("Please wait while exploring and visualizing materials...")
    
    # Initialize crystal-toolkit
    try:
        import crystal_toolkit.helpers.jupyter as ctj
        ctj.init_jupyter_mode()
    except:
        print("Note: For best visualization experience, ensure crystal-toolkit is properly initialized.")
    
    # Run the main exploration function
    oxide_df = explore_oxide_materials()
    
    # Optional: Uncomment to explore Si-O materials
    # si_o_df = explore_materials_by_chemsys("Si-O")
    
    print("\nAll explorations completed successfully!")

if __name__ == "__main__":
    main()

# Assignment Tasks:
# 1. Obtain your Materials Project API key and replace "YOUR_API_KEY" in the code
# 2. Run the code to explore oxide semiconductors using explore_oxide_materials()
# 3. Modify the code to explore a different class of materials:
#    - Use explore_materials_by_chemsys() with different chemical systems (e.g., "Si-N", "Ga-N")
#    - Or modify explore_oxide_materials() to search for different elements
# 4. Add a new visualization to explore different relationships between properties
# 5. Find the material with the highest band gap and visualize its structure
# 6. Write a brief report discussing the trends you observed in the data
#    - Include screenshots of your visualizations
#    - Discuss structure-property relationships

# Example of how to explore nitride semiconductors:
# To use this example, copy the code below and create a new function in this file
#
# def explore_nitride_materials():
#     with MPRester(API_KEY) as mpr:
#         # Query for nitride semiconductors
#         docs = mpr.materials.summary.search(
#             elements=["N"],
#             band_gap=(0.5, 6.0),
#             fields=[
#                 "material_id", 
#                 "formula_pretty", 
#                 "structure", 
#                 "band_gap", 
#                 "density", 
#                 "energy_above_hull", 
#                 "formation_energy_per_atom",
#                 "volume", 
#                 "symmetry"  # This gives access to crystal system information
#             ]
#         )
#         
#         # Then process results similar to explore_oxide_materials()
#         # Extract data to a list of dictionaries
#         data = []
#         for doc in docs:
#             try:
#                 crystal_system = doc.symmetry.crystal_system if hasattr(doc, 'symmetry') else "Unknown"
#             except:
#                 crystal_system = "Unknown"
#                 
#             data.append({
#                 "material_id": doc.material_id,
#                 "formula": doc.formula_pretty,
#                 "band_gap": doc.band_gap,
#                 "density": doc.density,
#                 "stability": doc.energy_above_hull,
#                 "formation_energy": doc.formation_energy_per_atom,
#                 "volume": doc.volume,
#                 "crystal_system": crystal_system,
#                 "structure": doc.structure
#             })
#         
#         # Convert to DataFrame and continue with visualization